<a href="https://colab.research.google.com/github/rtphokie/AstronomyDaysISS/blob/main/ISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skyfield
from skyfield.api import load, wgs84
import pytz
import pylab as pl
import matplotlib.pyplot as plt
ts = load.timescale()
horizon=10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.9/453.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 5.6 MB/s eta 0:00:00


[space station orbital elements (NORAD)](http://celestrak.com/NORAD/elements/stations.txt)


In [2]:
stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

[#################################] 100% stations.txt


Loaded 30 satellites


https://upload.wikimedia.org/wikipedia/commons/2/24/ISS_diagram.png

In [3]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']

print(satellite.epoch.utc_jpl())
print(satellite)

A.D. 2024-Apr-29 23:11:55.6388 UTC
ISS (ZARYA) catalog #25544 epoch 2024-04-29 23:11:56 UTC


Where is the ISS? [NASA Spot the Station](https://spotthestation.nasa.gov/tracking_map.cfm)

In [4]:
t = ts.now()
geocentric = satellite.at(t)
subpoint = wgs84.subpoint(geocentric)
print(f'Latitude:  {subpoint.latitude.degrees:.2f}')
print(f'Longitude: {subpoint.longitude.degrees:.2f}')
print(f"Elevation {int(subpoint.elevation.km)} (km) {int(subpoint.elevation.km*0.621371)} (mi)")
print(f"https://earth.google.com/web/search/{subpoint.latitude.degrees:.2f},{subpoint.longitude.degrees:.2f}/@47.13588096,47.3759601,-4649.26594136a,12274433.30377579d,35y,0h,0t,0r/data=Ck8aJRIfGY_C9ShcT0dAIR6F61G4nklAKgs0Ni42Miw1MS4yNBgCIAEiJgokCSNpsjq36UZAEVAjwaN0RkJAGZpZdCkAsFnAIbZNPnqeYV_AOgMKATA")


Latitude:  30.46
Longitude: 0.64
Elevation 415 (km) 258 (mi)
https://earth.google.com/web/search/30.46,0.64/@47.13588096,47.3759601,-4649.26594136a,12274433.30377579d,35y,0h,0t,0r/data=Ck8aJRIfGY_C9ShcT0dAIR6F61G4nklAKgs0Ni42Miw1MS4yNBgCIAEiJgokCSNpsjq36UZAEVAjwaN0RkJAGZpZdCkAsFnAIbZNPnqeYV_AOgMKATA


In [5]:
local_tz = pytz.timezone('US/Eastern')
local_lat=35.7796
local_lng=-78.6382
print(f"https://earth.google.com/web/search/{local_lat},{local_lng}/@{local_lat},{local_lng},1118.53612974a,602.64657559d,35y,0h,0t,0r/data=ClcaLRInGQGfH0YIS0VAISV1ApoIn1zAKhM0Mi41ODYxOSwgLTExNC40ODQ5GAIgASImCiQJSFNn4ihMRUARd6Kq1OtJRUAZCPZlHtqdXMAhURrod2ugXMA6AwoBMA")

local_coords = wgs84.latlon(local_lat,local_lng, elevation_m=1100)

https://earth.google.com/web/search/35.7796,-78.6382/@35.7796,-78.6382,1118.53612974a,602.64657559d,35y,0h,0t,0r/data=ClcaLRInGQGfH0YIS0VAISV1ApoIn1zAKhM0Mi41ODYxOSwgLTExNC40ODQ5GAIgASImCiQJSFNn4ihMRUARd6Kq1OtJRUAZCPZlHtqdXMAhURrod2ugXMA6AwoBMA


In [6]:
difference = satellite - local_coords
topocentric = difference.at(t)
alt, az, distance = topocentric.altaz()
print(f"alt {alt}, az {az} distance {int(distance.km)} (km) {int(distance.km*0.621371):,} (mi)")

alt -29deg 31' 58.1", az 69deg 25' 51.3" distance 7069 (km) 4,392 (mi)


ISS Passes [Heavens Above](https://www.heavens-above.com/ISS_3D.aspx)

In [29]:
t0 = ts.utc(2024, 4, 30)
t1 = ts.utc(2024, 5, 5)
t, events = satellite.find_events(local_coords, t0, t1, altitude_degrees=horizon)
for ti, event in zip(t, events):
    name = ('rise above 10°', 'culminate', 'set below 10°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2024 Apr 30 04:47:02 rise above 10°
2024 Apr 30 04:49:20 culminate
2024 Apr 30 04:51:39 set below 10°
2024 Apr 30 06:22:42 rise above 10°
2024 Apr 30 06:25:49 culminate
2024 Apr 30 06:28:56 set below 10°
2024 Apr 30 11:18:19 rise above 10°
2024 Apr 30 11:19:33 culminate
2024 Apr 30 11:20:46 set below 10°
2024 Apr 30 12:53:27 rise above 10°
2024 Apr 30 12:56:46 culminate
2024 Apr 30 13:00:03 set below 10°
2024 May 01 05:33:20 rise above 10°
2024 May 01 05:36:38 culminate
2024 May 01 05:39:57 set below 10°
2024 May 01 07:12:33 rise above 10°
2024 May 01 07:13:50 culminate
2024 May 01 07:15:07 set below 10°
2024 May 01 12:04:28 rise above 10°
2024 May 01 12:07:34 culminate
2024 May 01 12:10:40 set below 10°
2024 May 01 13:41:43 rise above 10°
2024 May 01 13:44:03 culminate
2024 May 01 13:46:22 set below 10°
2024 May 02 04:44:14 rise above 10°
2024 May 02 04:47:29 culminate
2024 May 02 04:50:45 set below 10°
2024 May 02 06:22:10 rise above 10°
2024 May 02 06:24:25 culminate
2024 May 02 06:

In [30]:
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']
def sun_pos(t, topopos):
  obs = earth + local_coords
  astrometric = obs.at(t).observe(sun)
  apparent = obs.at(t).observe(sun).apparent()
  alt, az, dist = apparent.altaz()
  return alt, az, dist

In [31]:
for ti, event in zip(t, events):
    name = (f'rise above {horizon}°', 'highest point', f'set below {horizon}°')[event]
    sunalt, sunaz, sundist = sun_pos(ti, local_coords)
    if -12 <= sunalt.degrees <= -6:
      print(ti.utc_jpl(), name)

A.D. 2024-May-04 09:38:29.9756 UTC rise above 10°
A.D. 2024-May-04 09:39:17.3735 UTC highest point
A.D. 2024-May-04 09:40:04.5971 UTC set below 10°


In [32]:

t, events = satellite.find_events(local_coords, t0, t1, altitude_degrees=horizon)
for ti, event in zip(t, events):
    name = (f'rise above {horizon}°', 'highest point', f'set below {horizon}°')[event]
    sunalt, sunaz, sundist = sun_pos(ti, local_coords)
    if -12 <= sunalt.degrees <= -6:
      print(ti.astimezone(local_tz).strftime('%a %b %d %-I:%M:%S %p'), name)
      if event ==2:
        print()

Sat May 04 5:38:29 AM rise above 10°
Sat May 04 5:39:17 AM highest point
Sat May 04 5:40:04 AM set below 10°

